<a href="https://colab.research.google.com/github/JonNData/used-car-predictor/blob/master/notebooks/cl_epa_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import re
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 

In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Go to link, sign in, and paste in result

In [3]:
file_list = drive.ListFile({'q': "'1kE03N5LwmWDDsDescCPmlQxGsY7ZKEnT' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: Weekly_U.S._All_Grades_All_Formulations_Retail_Gasoline_Prices, id: 1cmGfdjaAd7RXYa_9S8RcraEDBA1fJKoKzB17KcaEo7Y
title: Weekly_U.S._All_Grades_All_Formulations_Retail_Gasoline_Prices.csv, id: 1pBHxMMMbUXZfFhNj5c2KqFq_dm4EkYOK
title: vehicles.csv, id: 12aa_5F_dECaFGjQR5KYZwToAqhHouUNi
title: carfullspecs.csv, id: 1Y9oQ0huvw3LYnbafa-AM7Ig5cOK95DkD
title: vehicles_cl.csv, id: 1BKZfd5WBLkryOSWI6t4g50bloTD8P6f3


In [0]:
downloaded = drive.CreateFile({'id':"1BKZfd5WBLkryOSWI6t4g50bloTD8P6f3"})  
downloaded.GetContentFile('vehicles_cl.csv')       

In [5]:
df_orig = pd.read_csv('vehicles_cl.csv')
df_orig

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7088746062,https://greensboro.craigslist.org/ctd/d/cary-2...,greensboro,https://greensboro.craigslist.org,10299,2012.0,acura,tl,NaN,NaN,gas,90186.0,clean,automatic,19UUA8F22CA003926,NaN,NaN,other,blue,https://images.craigslist.org/01414_3LIXs9EO33...,2012 Acura TL Base 4dr Sedan Offered by: B...,NaN,nc,35.7636,-78.7443
1,7088745301,https://greensboro.craigslist.org/ctd/d/bmw-3-...,greensboro,https://greensboro.craigslist.org,0,2011.0,bmw,335,NaN,6 cylinders,gas,115120.0,clean,automatic,NaN,rwd,NaN,convertible,blue,https://images.craigslist.org/00S0S_1kTatLGLxB...,BMW 3 Series 335i Convertible Navigation Dakot...,NaN,nc,NaN,NaN
2,7088744126,https://greensboro.craigslist.org/cto/d/greens...,greensboro,https://greensboro.craigslist.org,9500,2011.0,jaguar,xf,excellent,NaN,gas,85000.0,clean,automatic,NaN,NaN,NaN,NaN,blue,https://images.craigslist.org/00505_f22HGItCRp...,2011 jaguar XF premium - estate sale. Retired ...,NaN,nc,36.1032,-79.8794
3,7088743681,https://greensboro.craigslist.org/ctd/d/cary-2...,greensboro,https://greensboro.craigslist.org,3995,2004.0,honda,element,NaN,NaN,gas,212526.0,clean,automatic,5J6YH18314L006498,fwd,NaN,SUV,orange,https://images.craigslist.org/00E0E_eAUnhFF86M...,2004 Honda Element LX 4dr SUV Offered by: ...,NaN,nc,35.7636,-78.7443
4,7074612539,https://lincoln.craigslist.org/ctd/d/gretna-20...,lincoln,https://lincoln.craigslist.org,41988,2016.0,chevrolet,silverado k2500hd,NaN,NaN,gas,NaN,clean,automatic,1GC1KWE85GF266427,NaN,NaN,NaN,NaN,https://images.craigslist.org/00S0S_8msT7RQquO...,"Shop Indoors, Heated Showroom!!!www.gretnaauto...",NaN,ne,41.1345,-96.2458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539754,7076296329,https://syracuse.craigslist.org/cto/d/mottvill...,syracuse,https://syracuse.craigslist.org,7950,2008.0,rover,NaN,excellent,8 cylinders,gas,NaN,clean,automatic,NaN,4wd,NaN,NaN,NaN,https://images.craigslist.org/00T0T_86RgcbSIhT...,Selling my 2008 Range Rover - it's an extra ve...,NaN,ny,42.9506,-76.4254
539755,7088752165,https://greensboro.craigslist.org/ctd/d/cary-2...,greensboro,https://greensboro.craigslist.org,13995,2016.0,nissan,frontier,NaN,NaN,gas,97545.0,clean,automatic,1N6AD0CW7GN748778,4wd,NaN,pickup,silver,https://images.craigslist.org/00R0R_2FiHQj8mDm...,2016 Nissan Frontier SV V6 4x4 4dr King Cab 6....,NaN,nc,35.7636,-78.7443
539756,7088749883,https://greensboro.craigslist.org/ctd/d/cary-2...,greensboro,https://greensboro.craigslist.org,10995,2010.0,bmw,335i,NaN,NaN,gas,80328.0,clean,automatic,WBAWB7C58AP049201,NaN,compact,coupe,black,https://images.craigslist.org/00v0v_3sp1G5hNhk...,2010 BMW 3 Series 335i 2dr Coupe Offered b...,NaN,nc,35.7636,-78.7443
539757,7088747986,https://greensboro.craigslist.org/ctd/d/cary-2...,greensboro,https://greensboro.craigslist.org,6495,2015.0,dodge,dart,NaN,NaN,other,83521.0,clean,manual,1C3CDFAA5FD252294,fwd,NaN,sedan,red,https://images.craigslist.org/00101_6orUUSDpdh...,2015 Dodge Dart SE 4dr Sedan Offered by: B...,NaN,nc,35.7636,-78.7443


In [68]:
df_cl = df_orig.copy()
df_cl['price'] = df_cl['price'].replace(0,np.NaN)
df_cl = df_cl.dropna(subset=['price'])
df_cl = df_cl[(df_cl['price'] >= np.percentile(df_cl['price'], 1)) &
          (df_cl['price'] <= np.percentile(df_cl['price'], 99))]
df_cl['price'].describe()

count    485223.000000
mean      13284.053765
std        9934.931696
min         199.000000
25%        5500.000000
50%       10800.000000
75%       18500.000000
max       52000.000000
Name: price, dtype: float64

In [0]:
df_cl = df_cl.dropna(subset=['year', 'manufacturer', 'model', 'year'])

In [8]:
df_cl.isnull().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer         0
model                0
condition       188861
cylinders       180060
fuel              2578
odometer         77101
title_status      2090
transmission      3146
vin             190182
drive           128460
size            313761
type            122327
paint_color     142543
image_url            0
description          1
county          458631
state                0
lat               3617
long              3617
dtype: int64

In [9]:
df_cl.shape

(458631, 25)

In [70]:
df_cl.model.sample(50)

346159                        cavalier
479335                          camaro
394087             sequoia limited 4x4
20241                            f-150
297601                        traverse
207426                            1500
429689                     sierra 1500
229371                             ltd
318779                          dakota
293171                silverado 3500hd
57577                            quest
517158    sierra 3500hd available wifi
151361                 camry hybrid le
356896                      tacoma 4x4
431818                   grand marquis
343899                        lacrosse
47778               silverado 1500 z71
439351    2 series 230i convertible 2d
449374                           f-150
136778                         c-class
218868                       silverado
34848                        f-150 xlt
327437                        veloster
438211                          murano
63213                           rx 350
441675                   

In [0]:
# EPA 
downloaded = drive.CreateFile({'id':"12aa_5F_dECaFGjQR5KYZwToAqhHouUNi"})  

downloaded.GetContentFile('vehicles.csv')  

In [13]:
df_epa = pd.read_csv('vehicles.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
df_epa.model.sample(50)

35197       M8 Competition Convertible
26996                   Genesis R Spec
18686                              600
7329                              540i
8482                              Echo
21519           Land Cruiser Wagon 4WD
34909                         Ghibli S
15973                Silverado K15 4WD
38908                     Hardbody 2WD
42012                          Prelude
2077                               XJ6
39473                  Wetl Testarossa
15395                            380SL
5225                          RAV4 2WD
18190               Cooper Convertible
30712                          Sonic 5
16245                328ci Convertible
3029                            Accord
17710    Van 1500 AWD Conversion Cargo
13823                       Armada 2WD
18487                          Skyhawk
39240                   B350 Wagon 2WD
4330                         Rodeo 2WD
22562                         Edge AWD
26774                     E350 Van FFV
20842                    

In [21]:
df_epa.make = df_epa.make.str.lower()


0        alfa romeo
1           ferrari
2             dodge
3             dodge
4            subaru
            ...    
42248        subaru
42249        subaru
42250        subaru
42251        subaru
42252        subaru
Name: make, Length: 42253, dtype: object

In [0]:
df_epa.model = df_epa.model.str.lower()

## Fuzzy Wuzzy attemps

In [15]:
pip install fuzzywuzzy

In [24]:
from fuzzywuzzy import fuzz, process

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [0]:
# Start with epa, compare models where year and make is the same

# for row in df_epa:
#   if df_epa.year == df_cl.year:
#     print(fuzz.ratio(df_epa['model'][row], df_cl['model'][row]))



In [71]:
# Lets look at a simplified list

epa_models = sorted(df_epa['model'].unique())
cl_models = sorted(df_cl.model.unique())


for model in epa_models:
  for x  in range(len(cl_models)):
    print(fuzz.ratio(model, cl_models[x]))

Streaming output truncated to the last 5000 lines.
18
18
32
21
18
18
44
24
27
29
28
30
27
30
24
21
24
26
17
34
29
25
27
29
18
32
21
17
18
20
35
19
24
30
21
24
36
17
34
41
35
28
26
29
29
32
30
13
29
42
32
27
35
24
22
19
21
28
25
43
27
29
30
15
27
29
26
24
29
28
42
52
10
15
11
26
29
28
17
26
26
27
24
26
26
30
22
22
23
18
16
25
26
15
24
41
22
26
36
20
29
27
14
25
26
27
9
9
17
22
35
31
32
38
30
21
32
21
25
25
35
23
25
35
30
35
30
35
31
44
26
25
35
39
24
32
31
32
38
27
27
20
20
20
26
18
27
31
14
9
20
17
7
24
15
28
17
16
7
16
12
16
16
16
7
18
12
10
18
9
16
13
18
17
10
13
8
8
19
9
10
10
15
15
10
15
14
20
31
10
23
28
29
7
25
12
17
20
9
22
10
9
17
11
7
5
13
19
18
17
12
15
8
7
11
7
11
9
15
14
10
15
8
9
8
11
5
16
6
20
23
7
16
18
16
16
17
12
16
16
16
17
7
6
18
12
37
9
8
29
13
44
6
7
12
18
12
7
6
17
11
9
17
7
31
14
14
15
19
21
20
15
15
15
14
24
7
12
12
12
12
19
10
10
18
28
11
12
12
9
11
18
24
29
12
17
24
16
30
29
27
26
31
16
19
17
31
22
17
10
8
26
17
20
21
10
11
48
44
24
17
9
22
28
7
8
7
14
18
22
1

KeyboardInterrupt: ignored

In [62]:
df_epa['model'].nunique(), df['model'].nunique()

(4186, 26904)

In [113]:
epa_models[2888]

'q60 red sport awd'

In [114]:
grand_cherokee_ratios = []
for model in cl_models:
  grand_cherokee_ratios.append(fuzz.ratio(epa_models[2888], model))
max(grand_cherokee_ratios)

84

In [115]:
grand_cherokee_ratios.index(84)

18845

In [116]:
cl_models[18845]

'q50 red sport 400 awd'

In [0]:
# Input Make model Year get a prediction?
sample_model = 'sivics'

cl_models = sorted(df_cl.model.unique())

def match_models(sample_model):
  model_ratios = []
  for model in cl_models:
    model_ratios.append(fuzz.ratio(sample_model, model))
  max_match = max(model_ratios)
  index_of_match = model_ratios.index(max_match)
  return cl_models[index_of_match]

In [126]:
match_models('Range Rover Evoque')

'range evoque'